In [10]:
from bs4 import BeautifulSoup as soup
from requests import get
import pandas as pd
import requests
import time
import re

In [11]:
latitude=[]
longitude=[]
locality_score=[]
age_of_property=[]
power_backup=[]
Lift_availability=[]
furniture_status=[]
Carpet_area=[]
bathrooms=[]
building_status=[]
facing=[]
title=[]
location=[]
price=[]
price_per_sqft=[]
area_in_sqft=[]
construction_status=[]
bhk=[]
property_type=[]

In [12]:
requests.get('https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?budget=,', verify=False)

C:\Users\admin\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<Response [200]>

In [13]:
%%time

n_pages = 0
for page in range(1678,1681):  #The no. of pages you wanted to scrap considering pages available in the url.
    n_pages += 1
    url = 'https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?budget=,'+'&page='+str(page)
    # This for gets us into next page after every iteration.
    r = ''
    while r == '':
        try:
            r = requests.get(url)
            break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            print("ZZzzzz...")
            time.sleep(5)
            print("Was a nice sleep, now let me continue...")
            continue # Here r is response data.
    page_html = soup(r.text, 'html.parser') 
    house_containers = page_html.find_all("li",class_="cardholder") # Class containing overall data of a property.
    for data in house_containers:
        type_=data.find_all("a",class_="typelink") # Class containing title name of a property.
        for i in type_:
            try:
                d=i.text
                title.append(d)
                try:
                    l=d.split()[2]
                    if l=='Independent':
                        w=d.split()[3]
                        w1=l+w
                        property_type.append(w1)
                    else:  
                        property_type.append(l)
                except:
                    property_type.append(d)
                a=d.split()[0]
                if a=='Residential':
                    bhk.append('0')
                else:
                    bhk.append(a)
            except:
                bhk.append(0)   

##########################################################################
        inner_url=data.find_all("meta") 
        try:
            b=inner_url[0]
            c=str(b)
            x = re.findall('"([^"]*)"',c)
            y=x[0]
            response = requests.get(y)
            data1=soup(response.text)
        except IndexError:
            continue

##########################################################################
        h_c12=data1.find_all("meta",itemprop="latitude")
        c12=str(h_c12)
        try:
            x2 = re.findall('"([^"]*)"',c12)
            s=float(x2[0])
            latitude.append(s)
        except IndexError:
            latitude.append('Not found')
##########################################################################
        h_c13=data1.find_all("meta",itemprop="longitude")
        c13=str(h_c13)
        try:  
            x3 = re.findall('"([^"]*)"',c13)
            s1=float(x3[0])
            longitude.append(s1) 
        except IndexError:
            longitude.append('Not found') 
##########################################################################
        h_c3=data1.find_all("div",class_="score css-178zeg4")
        try: 
            sc=h_c3[0].text
            locality_score.append(sc)
        except IndexError:
            locality_score.append("not found") 
##########################################################################
        h_c4=data1.find_all("div",class_="desc")
        s=str(h_c4)
        result=re.findall(r'\d+[ /]\by\w+',s)
        if result==[]:
            age_of_property.append("not found")   
        else:
            f1=result[0].split()[0]
            age_of_property.append(f1)
##########################################################################
        h_c5=data1.find_all("div",class_="css-3x3xre")
        try:
            a=0
            n=[] 
            for i in h_c5:
                j=h_c5[a].text
                a+=1
                n.append(j)
            if 'Power Backup'or 'Power Backup Available' in n:
                power_backup.append("yes") 
            else:
                power_backup.append("no")
        except:
            power_backup.append("not found")
        n.clear()
################################################################################
        h_c5_1=data1.find_all("div",class_="css-3x3xre")
        try:
            z=0
            m=[] 
            for i in h_c5_1:
                j=h_c5[z].text
                z+=1
                m.append(j)
            if 'Lift Available'or 'Lift' in m:
                Lift_availability.append("yes") 
            else:
                Lift_availability.append("no")
        except:
            power_backup.append("not found")
        m.clear()
##########################################################################
        h_c6=data1.find_all("td",id="Status")
        try: 
            f=h_c6[1].text
            furniture_status.append(f)
        except IndexError:
            furniture_status.append("not found") 
##########################################################################
        h_c7=data1.find_all("td",id="Carpet area")
        try: 
            a1=h_c7[0].text.split()[0]
            Carpet_area.append(int(a1))
        except IndexError:
            Carpet_area.append("not found") 
##########################################################################
        h_c8=data1.find_all("td",id="Bathrooms")
        try: 
            a2=h_c8[0].text
            bathrooms.append(int(a2))
        except IndexError:
            bathrooms.append(0) 
##########################################################################
        h_c9=data1.find_all("td",id="New/Resale")
        try:
            a3=h_c9[0].text
            building_status.append(a3)
        except IndexError:
            building_status.append("not found")
##########################################################################
        h_c10=data1.find_all("td",id="Facing")
        try: 
            a4=h_c10[0].text
            facing.append(a4)
        except IndexError:
            facing.append(0)
            
 ##############################################################################

        location_=data.find_all("a",class_="loclink")
        
        try:
            for i in location_:
                d=i.text
                r= d.split(',',)[0] # Splitting the obtained text and returning the first element of text.
                location.append(r)
        except:
            location.append("not found")
            
################################    PRICE    ######################################

        cost=data.find_all("td",class_="price")
        try:
            for i in cost:
                d=i.text
                if 'L' in d:
                    a = d.split()[0]
                    price.append(float(a)) # Type conversion of obtained data.
                elif 'Cr' in d:
                    b = d.split()[0]
                    price.append(float(b) * 100) # Coverting CRORES into LAKHS.
        except:  # If no value found in the class return value 0.
            e=0
            price.append(float(e)) 
                
############################   PRICE PER SQFT   ###################################

        rate_sqft=data.find_all('td',class_="lbl rate")
        
        
        try:
            for i in rate_sqft:
                d=i.text
                res= d.split('/',)[0]
                r=re.sub(",","" ,res) # Substitute no character in the place of ,(comma).
                price_per_sqft.append(r)
        except:
            price_per_sqft.append(0)
            
############################   AREA IN SQFT    ####################################

        area_sqft=data.find_all('td',class_="size")
        for i in area_sqft:
            d=i.text
            area_in_sqft.append(d)
            
##########################   BUILDING STATUS    ###################################

        building_status_=data.find_all("td",class_="val")
        for i in building_status_:
            d=i.text
            construction_status.append(d)    

print('You scraped {} pages containing {} properties.'.format(n_pages, len(title)))

You scraped 3 pages containing 60 properties.
Wall time: 1min 34s


In [14]:
df=pd.DataFrame()

In [15]:
len(bhk),len(property_type),len(age_of_property),len(power_backup),len(latitude),len(locality_score),len(longitude),len(Lift_availability),len(furniture_status),len(Carpet_area),len(bathrooms),len(building_status),len(facing),len(title),len(location),len(price),len(price_per_sqft),len(area_in_sqft),len(construction_status)

(60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60)

In [16]:
df['type']=title
df['latitude']=latitude
df['longitude']=longitude
df['locaLity score']=locality_score
df['age of property']=age_of_property
df['power backup']=power_backup
df['lift']=Lift_availability
df['furninture status']=furniture_status
df['carpet area']=Carpet_area
df['bathrooms']=bathrooms
df['status of building']=building_status
df['facing']=facing
df['location']=location
df['price(L)']=price
df['rate persqft']=price_per_sqft
df['area insqft']=area_in_sqft
df['construction status']=construction_status
df['property type']=property_type
df['Bhk']=bhk

In [17]:
df.shape

(60, 19)

In [18]:
df

,type,latitude,longitude,locaLity score,age of property,power backup,lift,furninture status,carpet area,bathrooms,status of building,facing,location,price(L),rate persqft,area insqft,construction status,property type,Bhk
0,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,East,Kadthal,23.50,522,4500,New,Residential Plot,0
1,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,North,Kadthal,20.21,522,3870,New,Residential Plot,0
2,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,North,Kadthal,14.06,411,3420,New,Residential Plot,0
3,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,North,Kadthal,21.71,522,4158,New,Residential Plot,0
4,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,East,Kadthal,19.24,411,4680,New,Residential Plot,0
5,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,West,Kadthal,16.35,411,3978,New,Residential Plot,0
6,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,West,Kadthal,13.87,411,3375,New,Residential Plot,0
7,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,North,Kadthal,13.50,411,3285,New,Residential Plot,0
8,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,West,Kadthal,16.68,522,3195,New,Residential Plot,0
9,Residential Plot,16.988502,78.496834,7.7,not found,yes,yes,not found,not found,0,New,North,Kadthal,16.09,411,3915,New,Residential Plot,0
